Homework is at the bottom of this notebook below the course material

# Course

In [2]:
from platform import python_version

print(python_version())

3.10.14


## Get minsearch.py file and use it as package

In [3]:
#!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

In [4]:
import minsearch

## Get FAQ parser notebook

In [5]:
#!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/parse-faq.ipynb

## Get documents.json file generated after FAQ parsing

In [6]:
#!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

In [7]:
import json

In [8]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [9]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [10]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [11]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

```keyword_fields``` operate like and SQL Query : ```SELECT * WHERE course = 'data-engineering-zoomcamp';```

In [12]:
q = 'the course has already started, can I still enroll?'

In [13]:
index.fit(documents)

```boost``` can be used when we think that one text field is more important than the other in the final result

In [14]:
boost = {
    'question': 3.0, # 3 times more important
    'section': 0.5, # 0.5 less important 
}

filter_d = {
    'course': 'data-engineering-zoomcamp'
}

index.search(
    query=q,
    filter_dict=filter_d,
    boost_dict=boost,
    num_results=5
)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [15]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [16]:
api_key = os.environ["MISTRAL_API_KEY"]
model = "mistral-large-latest"

In [17]:
client = MistralClient(api_key=api_key)

chat_response = client.chat(
    model=model,
    messages=[ChatMessage(role="user", content=q)]
)

print(chat_response.choices[0].message.content)

That depends on the policy of the course provider. Some courses allow late enrollment, while others do not. I would recommend reaching out to the course provider directly to ask if late enrollment is allowed and if there are any restrictions or additional requirements for joining the course after it has started. They should be able to provide you with the most accurate information and help you determine if it's still possible for you to enroll.


In [18]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [19]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [20]:
def llm(prompt):
    chat_response = client.chat(
        model=model,
        messages=[ChatMessage(role="user", content=prompt)]
    )
    
    return chat_response.choices[0].message.content

In [21]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [22]:
query = "how to enroll in the course?"

In [23]:
rag(query)

"To enroll in the course, you need to register before the course starts using the registration link provided. Additionally, it's recommended that you subscribe to the course public Google Calendar, join the course Telegram channel for announcements, and register in DataTalks.Club's Slack and join the channel. The exact start date of the course is 15th Jan 2024 at 17h00. Even if you miss the start date, you can still join the course, but be aware of the deadlines for turning in the final projects."

## Switching the minsearch engine with Elasticsearch

Running ElasticSearch with Docker
```bash
docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

Index settings:
```json
{
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
```

Query:
```json
{
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}
```
We use "type": "best_fields"

### Elastic Search 

This document contains useful things about Elasticsearch

## `multi_match` Query in Elasticsearch

The `multi_match` query is used to search for a given text across multiple fields in an Elasticsearch index.

It provides various types to control how the matching is executed and scored. 

There are multiple types of `multi_match` queries:

- `best_fields`: Returns the highest score from any one field.
- `most_fields`: Combines the scores from all fields.
- `cross_fields`: Treats fields as one big field for scoring.
- `phrase`: Searches for the query as an exact phrase.
- `phrase_prefix`: Searches for the query as a prefix of a phrase.


## `best_fields`

The `best_fields` type searches each field separately and returns the highest score from any one of the fields.

This type is useful when you want to find documents where at least one field matches the query well.


```json
{
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": "How do I run docker on Windows?",
                    "fields": ["question", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}
```

## `most_fields`

The `most_fields` type searches each field and combines the scores from all fields.

This is useful when the relevance of a document increases with more matching fields.

```json
{
    "multi_match": {
        "query": "How do I run docker on Windows?",
        "fields": ["question^4", "text"],
        "type": "most_fields"
    }
}
```

## `cross_fields`

The `cross_fields` type treats fields as though they were one big field.

It is suitable for cases where you have fields representing the same text in different ways, such as synonyms.

```json
{
    "multi_match": {
        "query": "How do I run docker on Windows?",
        "fields": ["question", "text"],
        "type": "cross_fields"
    }
}
```

## `phrase`

The `phrase` type looks for the query as an exact phrase within the fields.

It is useful for exact match searches.

```json
{
    "multi_match": {
        "query": "How do I run docker on Windows?",
        "fields": ["question", "text"],
        "type": "phrase"
    }
}
```

## `phrase_prefix`

The `phrase_prefix` type searches for documents that contain the query as a prefix of a phrase.

This is useful for autocomplete or typeahead functionality.


```json
{
    "multi_match": {
        "query": "How do I run docker on Windows?",
        "fields": ["question", "text"],
        "type": "phrase_prefix"
    }
}
```

In [24]:
from elasticsearch import Elasticsearch

In [25]:
es_client = Elasticsearch('http://localhost:9200') 

In [30]:
query = 'I just disovered the course. Can I still join it?'

In [31]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"], # ^3 means that fields is 3 times more important
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [32]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [33]:
rag(query)

"Yes, you can still join the course even if it has already started. You're eligible to submit homeworks, but please be aware that there will be deadlines for turning in the final projects. Try not to leave everything for the last minute. You don't need to register to start learning and submitting homework, as it's not checked against any registered list. Registration is just to gauge interest before the start date."

# Homework

### Question 1

In [26]:
!curl localhost:9200

{
  "name" : "78d984fda2bd",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "u56znaAsR9GsF3GJOQEWAA",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [28]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

### Question 2

In [29]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████| 948/948 [00:10<00:00, 93.63it/s]


### Question 3

In [34]:
query = "How do I execute a command in a running docker container?"

In [36]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)
response['hits']['hits'][0]["_score"]

84.050095

### Question 4

In [38]:
search_query_q4 = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response_q4 = es_client.search(index=index_name, body=search_query)
response['hits']['hits'][2]

{'_index': 'course-questions',
 '_id': 'umzsTpABPs3o_u7MQf3j',
 '_score': 49.938507,
 '_source': {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from a different folder into docker container’s working directory?',
  'course': 'machine-learning-zoomcamp'}}

### Question 5

In [40]:
result_docs_q4 = []

for hit in response['hits']['hits']:
    result_docs_q4.append(hit['_source'])

In [42]:
prompt_template_q4 = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

context_q4 = ""

for doc in result_docs_q4:
    context_q4 += f"Q: {doc['question']}\nA: {doc['text']}\n\n"

prompt_q4 = prompt_template_q4.format(question=query, context=context_q4).strip()

In [44]:
len(prompt_q4)

1462

### Question 6

In [59]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(prompt_q4)
len(tokens)

322

### Question 7

In [43]:
chat_response_q4 = client.chat(
    model=model,
    messages=[ChatMessage(role="user", content=prompt_q4)]
)
    
chat_response_q4.choices[0].message.content

"To execute a command in a running Docker container, you first need to find the container's ID. You can do this by running the command `docker ps`, which will list all running containers along with their IDs. Once you have the container's ID, you can use the `docker exec` command to run a command in that container. The syntax for this command is `docker exec -it <container-id> bash`, where `<container-id>` is the ID of the container in which you want to execute the command. This will launch a bash shell in the running container, allowing you to execute commands within it."